In [1]:
import scipy.io as sio  
import matplotlib.pyplot as plt  
import numpy as np  
import os
import tensorflow as tf
import matplotlib.image as mpimg
import matplotlib.patches as patches
import numpy
from PIL import Image

In [2]:
'''
def load_data():
    
    faces = np.empty((700, 165 * 120))
    label = np.zeros((700, 100))

    data=sio.loadmat('AR_database.mat')

    Tr_dataMatrix = data['Tr_dataMatrix'].astype('float32')/256
    Tr_sampleLabels = data['Tr_sampleLabels'].astype('float32')
    Tt_dataMatrix = data['Tt_dataMatrix'].astype('float32')/256
    Tt_sampleLabels = data['Tt_sampleLabels'].astype('float32')

    faces = np.empty((1400, 165 * 120))

    train_data = Tr_dataMatrix.T
    train_label = np.zeros((700, 100))
    for i in range(100):
        train_label[i * 7: (i + 1) * 7, i] = 1

    test_data = Tt_dataMatrix.T
    test_label = np.zeros((700, 100))
    for i in range(100):
         test_label[i * 7: (i + 1) * 7, i] = 1
    face = np.concatenate((train_data,test_data),axis=0)
    label = np.concatenate((train_label,test_label),axis=0)
    
    train_data = np.empty((1120, 165*120))
    train_label = np.zeros((1120, 100))
    vaild_data = np.empty((200, 165 * 120))
    vaild_label = np.zeros((200, 100))
    test_data = np.empty((200, 165 * 120))
    test_label = np.zeros((200, 100))
    
    
    for i in range(200):
        train_data[i * 5: i * 5 + 5] = face[i * 7: i * 7 + 5]
        train_label[i * 5: i * 5 + 5] = label[i * 7: i * 7 + 5]

        vaild_data[i] = face[i * 7 + 5]
        vaild_label[i] = label[i * 7 + 5]

        test_data[i] = face[i * 7 + 6]
        test_label[i] = label[i * 7 + 6]
        
    train_data = train_data.astype('float32')
    vaild_data = vaild_data.astype('float32')
    test_data = test_data.astype('float32')

    return [
        (train_data, train_label),
        (vaild_data, vaild_label),
        (test_data, test_label)
    ]
'''
def load_face():
    data=sio.loadmat('AR_buzhedang.mat')
    Tr_dataMatrix = data['O_Tt_DAT'].T.astype('float32')/256
    Tr_sampleLabels = data['trls'].astype('float32')
    dat=Tr_dataMatrix
    label = np.zeros((700, 100))
    for i in range(100):
        label[i * 7: (i + 1) * 7, i] = 1
        
    faces = np.empty((700, 57 * 47))
    
    for i in range(700):
        bbb = dat[:][i]
        grid = [[0]*57]*47
        bbb.resize((47,57))
        grid = bbb.T
        g2 = [[0]*2679]*1
        matrix_a = np.array(g2)
        matrix_a = matrix_a.astype('float32')
        g=0
        for j in range(57):
            for k in range(47):
                matrix_a[0][g]=grid[j][k]
                g=g+1
        faces[i][:]= matrix_a
        
    #print faces.shape
    label1 = np.zeros((200,40))
    for i in range(40):
        label1[i*5:(i+1)*5,i]=1
        
    label2 = np.zeros((40,40))
    for i in range(40):
        label2[i,i]=1
        

    train_data = np.empty((500, 57 * 47))
    train_label = np.zeros((500, 100))
    vaild_data = np.empty((100, 57 * 47))
    vaild_label = np.zeros((100, 100))
    test_data = np.empty((100, 57 * 47))
    test_label = np.zeros((100, 100))

    for i in range(100):
        train_data[i * 5: i * 5 + 5] = faces[i * 7+2: i * 7 + 7]
        train_label[i * 5: i * 5 + 5] = label[i * 7+2: i * 7 + 7]

        vaild_data[i] = faces[i * 7]
        vaild_label[i] = label[i * 7 + 5]

        test_data[i] = faces[i * 7+1 ]
        test_label[i] = label[i * 7 + 6]

    train_data = train_data.astype('float32')
    vaild_data = vaild_data.astype('float32')
    test_data = test_data.astype('float32')

    '''
    return [
        (train_data[0:200][:], train_label[0:200]),
        (vaild_data[0:40][:], vaild_label[0:40]),
        (test_data[0:40][:], test_label[0:40])
    ]
    '''
    return [
        (train_data[0:200][:], label1),
        (vaild_data[0:40][:], label2),
        (test_data[0:40][:], label2)
    ]



In [3]:
def convolutional_layer(data, kernel_size, bias_size, pooling_size):
    kernel = tf.get_variable("conv", kernel_size, initializer=tf.random_normal_initializer())
    bias = tf.get_variable('bias', bias_size, initializer=tf.random_normal_initializer())

    conv = tf.nn.conv2d(data, kernel, strides=[1, 1, 1, 1], padding='SAME')
    linear_output = tf.nn.relu(tf.add(conv, bias))
    pooling = tf.nn.max_pool(linear_output, ksize=pooling_size, strides=pooling_size, padding="SAME")
    return pooling

def linear_layer(data, weights_size, biases_size):
    weights = tf.get_variable("weigths", weights_size, initializer=tf.random_normal_initializer())
    biases = tf.get_variable("biases", biases_size, initializer=tf.random_normal_initializer())
    return tf.add(tf.matmul(data, weights), biases)

def convolutional_neural_network(data):
    # 根据类别个数定义最后输出层的神经元
    n_ouput_layer = 40

    kernel_shape1=[5, 5, 1, 32]
    kernel_shape2=[5, 5, 32, 64]
    full_conn_w_shape = [15 * 12 * 64, 1024]
    out_w_shape = [1024, n_ouput_layer]

    bias_shape1=[32]
    bias_shape2=[64]
    full_conn_b_shape = [1024]
    out_b_shape = [n_ouput_layer]

    data = tf.reshape(data, [-1, 57, 47, 1])

    # 经过第一层卷积神经网络后，得到的张量shape为：[batch, 29, 24, 32]
    with tf.variable_scope("conv_layer1") as layer1:
        layer1_output = convolutional_layer(
            data=data,
            kernel_size=kernel_shape1,
            bias_size=bias_shape1,
            pooling_size=[1, 2, 2, 1]
        )
    # 经过第二层卷积神经网络后，得到的张量shape为：[batch, 15, 12, 64]
    with tf.variable_scope("conv_layer2") as layer2:
        layer2_output = convolutional_layer(
            data=layer1_output,
            kernel_size=kernel_shape2,
            bias_size=bias_shape2,
            pooling_size=[1, 2, 2, 1]
        )
    with tf.variable_scope("full_connection") as full_layer3:
        # 讲卷积层张量数据拉成2-D张量只有有一列的列向量
        layer2_output_flatten = tf.contrib.layers.flatten(layer2_output)
        print("vec:=====:",layer2_output_flatten)
        layer3_output = tf.nn.relu(
            linear_layer(
                data=layer2_output_flatten,
                weights_size=full_conn_w_shape,
                biases_size=full_conn_b_shape
            )
        )
        print("vec:=====:",layer3_output)
        # layer3_output = tf.nn.dropout(layer3_output, 0.8)
    with tf.variable_scope("output") as output_layer4:
        output = linear_layer(
            data=layer3_output,
            weights_size=out_w_shape,
            biases_size=out_b_shape
        )
    print output.shape

    return output;

def train_facedata(dataset, model_dir,model_path):
    # train_set_x = data[0][0]
    # train_set_y = data[0][1]
    # valid_set_x = data[1][0]
    # valid_set_y = data[1][1]
    # test_set_x = data[2][0]
    # test_set_y = data[2][1]
    # X = tf.placeholder(tf.float32, shape=(None, None), name="x-input")  # 输入数据
    # Y = tf.placeholder(tf.float32, shape=(None, None), name='y-input')  # 输入标签

    batch_size = 40

    # train_set_x, train_set_y = dataset[0]
    # valid_set_x, valid_set_y = dataset[1]
    # test_set_x, test_set_y = dataset[2]
    train_set_x = dataset[0][0]
    train_set_y = dataset[0][1]
    valid_set_x = dataset[1][0]
    valid_set_y = dataset[1][1]
    test_set_x = dataset[2][0]
    test_set_y = dataset[2][1]

    X = tf.placeholder(tf.float32, [batch_size, 57 * 47])
    Y = tf.placeholder(tf.float32, [batch_size, 40])
    print("Y=",Y)

    predict = convolutional_neural_network(X)
    cost_func = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=Y))
    optimizer = tf.train.AdamOptimizer(1e-2).minimize(cost_func)

    # 用于保存训练的最佳模型
    saver = tf.train.Saver()
    #model_dir = './model'
    #model_path = model_dir + '/best.ckpt'
    with tf.Session() as session:
        # 若不存在模型数据，需要训练模型参数
        if not os.path.exists(model_path + ".index"):
            session.run(tf.global_variables_initializer())
            best_loss = float('Inf')
            for epoch in range(25):
                epoch_loss = 0
                for i in range((int)(np.shape(train_set_x)[0] / batch_size)):
                    x = train_set_x[i * batch_size: (i + 1) * batch_size]
                    y = train_set_y[i * batch_size: (i + 1) * batch_size]
                    _, cost = session.run([optimizer, cost_func], feed_dict={X: x, Y: y})
                    epoch_loss += cost

                print(epoch, ' : ', epoch_loss)
                if best_loss > epoch_loss:
                    best_loss = epoch_loss
                    if not os.path.exists(model_dir):
                        os.mkdir(model_dir)
                        print("create the directory: %s" % model_dir)
                    save_path = saver.save(session, model_path)
                    print("Model saved in file: %s" % save_path)

        # 恢复数据并校验和测试
        saver.restore(session, model_path)
        correct = tf.equal(tf.argmax(predict,1), tf.argmax(Y,1))
        valid_accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('valid set accuracy: ', valid_accuracy.eval({X: valid_set_x, Y: valid_set_y}))

        test_pred = tf.argmax(predict, 1).eval({X: test_set_x})
        test_true = np.argmax(test_set_y, 1)
        test_correct = correct.eval({X: test_set_x, Y: test_set_y})
        incorrect_index = [i for i in range(np.shape(test_correct)[0]) if not test_correct[i]]
        for i in incorrect_index:
            print('picture person is %i, but mis-predicted as person %i'
                %(test_true[i], test_pred[i]))
        #plot_errordata(incorrect_index)

#画出在测试集中错误的数据
def plot_errordata(error_index):
    data[2][0][:][error_index]
    pig=data.resize((57,47))
    plt.show(pig)

def main():
    dataset_path = "olivettifaces.gif"
    data = load_face()
    model_dir = './model'
    model_path = model_dir + '/best.ckpt'
    train_facedata(data, model_dir, model_path)

if __name__ == "__main__" :
    main()

('Y=', <tf.Tensor 'Placeholder_1:0' shape=(40, 40) dtype=float32>)
('vec:=====:', <tf.Tensor 'full_connection/Flatten/Reshape:0' shape=(40, 11520) dtype=float32>)
('vec:=====:', <tf.Tensor 'full_connection/Relu:0' shape=(40, 1024) dtype=float32>)
(40, 40)
(0, ' : ', 1798208.078125)
create the directory: ./model
Model saved in file: ./model/best.ckpt
(1, ' : ', 747435.1171875)
Model saved in file: ./model/best.ckpt
(2, ' : ', 334399.0625)
Model saved in file: ./model/best.ckpt
(3, ' : ', 142672.294921875)
Model saved in file: ./model/best.ckpt
(4, ' : ', 68015.68798828125)
Model saved in file: ./model/best.ckpt
(5, ' : ', 39425.1435546875)
Model saved in file: ./model/best.ckpt
(6, ' : ', 23843.647216796875)
Model saved in file: ./model/best.ckpt
(7, ' : ', 13712.248229980469)
Model saved in file: ./model/best.ckpt
(8, ' : ', 8168.864990234375)
Model saved in file: ./model/best.ckpt
(9, ' : ', 4292.5559539794922)
Model saved in file: ./model/best.ckpt
(10, ' : ', 2149.8984603881836)
Mod